# Metehan Ayhan - Datathon 2024 Project

Bu yarışma, her bir örnek verinin Değerlendirme Puanını doğru bir şekilde tahmin etmeyi amaçlamaktadır. Yarışmanın başarısını ölçmek için kullanılacak metrik ise RMSE olacaktır.

<a href='https://www.kaggle.com/competitions/datathon-2024/data' target=_blank>Data dosyasını burada bulabilirsiniz.</a>

### Gerekli Paketleri İmport Edelim

In [5]:
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows', None)

from sklearn.impute import KNNImputer
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

from sklearn.linear_model import Ridge, Lasso, ElasticNet

import warnings
warnings.filterwarnings('ignore')

import datetime

### Import Data

In [7]:
test=pd.read_csv('test_x.csv')
train=pd.read_csv('train.csv')

In [8]:
df = pd.concat([train, test], axis=0, ignore_index=True)

In [9]:
train['Degerlendirme Puani'].fillna(0, inplace=True)

### EDA - Exploratory Data Analysis

In [11]:
df.sample(5)

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
44488,2020,13.0,Erkek,1/13/01 0:00,Ankara,İstanbul,KÜTAHYA DUMLUPINAR ÜNİVERSİTESİ,Devlet,NaN,Hayır,Spor Bilimleri Fakültesi,1,3.00 - 3.50,NaN,Ankara Aziziye Anadolu Lisesi,NaN,Ankara,Özel,Sayısal,NaN,75 - 100,Hayır,NaN,NaN,Ortaokul,Hayır,NaN,Ortaokul,Evet,Kamu,3.0,Hayır,NaN,Evet,Diğer,Hayır,NaN,NaN,Hayır,NaN,Hayır,NaN,NaN,44488
23698,2016,68.0,Erkek,1996-06-16,Adana,Adana,Çukurova Üniversitesi,Devlet,NaN,Hayır,Maliye,3,2.50 ve altı,Hayır,İncirlik anadolu lisesi,NaN,Adana,Düz Lise,Eşit Ağırlık,NaN,NaN,Evet,Kyk,400,Eğitim Yok,Hayır,NaN,İlkokul Mezunu,Emekli,NaN,5.0,Evet,Sinerji,Evet,Diğer,Evet,Akp gençlik kolları,Hayır,Evet,NaN,Evet,Orta,NaN,23698
63930,2022,16.0,Erkek,16.11.1999 00:00,Adıyaman,Gaziantep,Kilis 7 Aralık Üniversitesi,Devlet,NaN,Hayır,İktisat,3,2.50 - 2.99,NaN,Açık öğretim lisesi,NaN,Gaziantep,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,İlkokul,Hayır,-,3.0,Hayır,NaN,Evet,Takım Oyuncusu,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,63930
72977,2023,NaN,Erkek,20.06.2002 00:00,İstanbul,İstanbul,Yıldız Teknik Üniversitesi,Devlet,NaN,Hayır,Kimya Mühendisliği,3,2.50 - 2.99,NaN,ihsan mermerci anadolu lisesi,NaN,İstanbul,Devlet,Sayısal,NaN,75 - 100,Evet,ibb bursu ve bir çok vakıftan başarı bursu alı...,1000₺ ve üstü,Lise,Evet,Özel Sektör,Üniversite,Hayır,-,1.0,Hayır,NaN,Hayır,-,Evet,AIESEC İstanbulda 6 aylık bir denemem oldu. Üy...,NaN,Hayır,-,Evet,NaN,NaN,7852
32130,2017,30.0,ERKEK,3/29/1998,Edirne,Istanbul,BOĞAZİÇİ ÜNİVERSİTESİ,DEVLET,NaN,Hayır,İnşaat Mühendisliği(İngilizce),1,ORTALAMA BULUNMUYOR,NaN,Edirne Süleyman Demirel Fen Lisesi,NaN,Edirne,Fen Lisesi,Sayısal,NaN,100-85,Hayır,NaN,NaN,LİSE,Hayır,NaN,LİSE,Evet,ÖZEL SEKTÖR,1.0,Hayır,NaN,Hayır,NaN,Hayır,NaN,Evet,Evet,NaN,Evet,İleri,NaN,32130


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76174 entries, 0 to 76173
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               76174 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   76005 non-null  object 
 3   Dogum Tarihi                                               75997 non-null  object 
 4   Dogum Yeri                                                 75383 non-null  object 
 5   Ikametgah Sehri                                            74137 non-null  object 
 6   Universite Adi                                             76042 non-null  object 
 7   Universite Turu                                            75919 non-null  object 
 8   Burslu

In [13]:
df.isnull().sum()

Basvuru Yili                                                     0
Degerlendirme Puani                                          11050
Cinsiyet                                                       169
Dogum Tarihi                                                   177
Dogum Yeri                                                     791
Ikametgah Sehri                                               2037
Universite Adi                                                 132
Universite Turu                                                255
Burslu ise Burs Yuzdesi                                      53734
Burs Aliyor mu?                                                  0
Bölüm                                                          231
Universite Kacinci Sinif                                       374
Universite Not Ortalamasi                                     2753
Daha Once Baska Bir Universiteden Mezun Olmus                48394
Lise Adi                                                      

### Filling Missing Data and Data Conversion

In [15]:
#CİNSİYET
df['Cinsiyet'].fillna(0, inplace=True)
df['Cinsiyet'] = df['Cinsiyet'].replace({
    'Erkek': 1,
    'ERKEK': 1,
    'Kadın': 2,
    'Belirtmek istemiyorum': 0
})

In [16]:
#DOĞUM TARİHİ
df['Dogum Tarihi'] = df['Dogum Tarihi'].str.split(' ').str[0]

def clean_date(date_str):
    if pd.isna(date_str):
        return date_str
    return re.sub(r'[.\-/]', ' ', str(date_str))

df['Dogum Tarihi'] = df['Dogum Tarihi'].apply(clean_date)
df['Dogum Tarihi'] = df['Dogum Tarihi'].str.replace('_', '9')

def parse_date(date_str):
    if pd.isna(date_str):
        return '2000', '01', '01' 
    
    parts = date_str.split()
    
    if len(parts[0]) == 4 and parts[0].isdigit():
        year = parts[0]
        month = parts[1] if len(parts) > 1 else '01'
        day = parts[2] if len(parts) > 2 else '01'
    elif len(parts) > 2 and len(parts[2]) == 4 and parts[2].isdigit():
        year = parts[2]
        month = parts[0] if len(parts) > 0 else '01'
        day = parts[1] if len(parts) > 1 else '01'
    else:
        if len(parts) == 3:
            year_part = parts[2]
            if len(year_part) == 2:
                year = '20' + year_part if int(year_part) <= 40 else '19' + year_part
            else:
                year = year_part
            month = parts[0]
            day = parts[1]
        elif len(parts) == 2:
            year_part = parts[1]
            year = '20' + year_part if int(year_part) <= 40 else '19' + year_part
            month = parts[0]
            day = '01'
        else:
            year = '2000' 
            month = '01'
            day = '01'
    
    return year, month, day

df[['yıl', 'ay', 'gün']] = df['Dogum Tarihi'].apply(lambda x: pd.Series(parse_date(x)))

df['Basvuru Yili'] = df['Basvuru Yili'].astype(int)
df['yıl'] = df['yıl'].astype(int)

df['yas'] = df['Basvuru Yili'] - df['yıl']
df.drop(columns=['ay', 'gün','yıl','Dogum Tarihi'], inplace=True)

In [17]:
# İKAMETGAH ŞEHRİ
df['Ikametgah Sehri'] = df['Ikametgah Sehri'].str.lower()
df['Ikametgah Sehri'].fillna(df['Ikametgah Sehri'].mode()[0], inplace=True)
def etiketle(text):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    if 'istanbul' in text or 'ankara' in text or 'izmir' in text or 'bursa' in text or 'antalya' in text or 'adana' in text or 'konya' in text or 'gaziantep' in text or 'kayseri' in text or 'mersin' in text or 'samsun' in text or 'sakarya' in text or 'eskişehir' in text or 'denizli' in text or 'kocaeli' in text or 'trabzon' in text or 'manisa' in text or 'balıkesir' in text or 'muğla' in text or 'hatay' in text or 'aydın' in text or 'zonguldak' in text or 'ordu' in text or 'giresun' in text or 'kırıkkale' in text or 'ağrı' in text or 'kırıkkale' in text or 'şanlıurfa' in text:
        return 3
    elif 'afyonkarahisar' in text or 'bilecik' in text or 'bolu' in text or 'çorum' in text or 'diyarbakır' in text or 'sivas' in text or 'tunceli' in text or 'bingöl' in text or 'ardahan' in text or 'kars' in text or 'bitlis' in text or 'hakkari' in text or 'van' in text or 'igdir' in text or 'şırnak' in text or 'karaman' in text or 'gümüşhane' in text or 'karabük' in text or 'osmaniye' in text or 'düzce' in text or 'burdur' in text or 'gölcük' in text or 'yalova' in text or 'bartın' in text or 'kastamonu' in text or 'sinop' in text or 'çankırı' in text or 'artvin' in text or 'rize' in text or 'bayburt' in text:
        return 4
    elif 'yurtdışı' in text or 'kktc' in text:
        return 1
    else:
        return 2

df['Ikametgah Sehri']= df['Ikametgah Sehri'].apply(etiketle)


In [18]:
# UNIVERSITE TURU
df['Universite Turu'].fillna(df['Universite Turu'].mode()[0], inplace=True)
df['Universite Turu'] = df['Universite Turu'].replace({
    'Devlet': 1, 
    'DEVLET': 1, 
    'Özel': 0, 
    'ÖZEL': 0
})

In [19]:
# UNIVERSITE ADI
def etiketle(text):
    if pd.isna(text):
        return 0
    text = str(text).lower()  
    if 'boğaziçi' in text or 'istanbul teknik' in text or 'odtü' in text or 'bilkent' in text or 'medipol' in text or 'bahçeşehir' in text or 'başkent' in text or 'yaşar' in text or 'beykent' in text or 'koç' in text or 'sabancı' in text or 'çankaya' in text or 'yeditepe' in text or 'hacettepe' in text or 'orta doğu teknik' in text or 'ankara üniversitesi' in text or 'gazi' in text or 'yıldız teknik' in text or 'akdeniz' in text or 'ege' in text or 'galatasaray' in text:
        return 4
    elif 'uludağ' in text or 'orhangazi' in text or 'türk hava kurumu' in text or 'kocaeli' in text or 'firat' in text or 'anadolu' in text or 'çukurova' in text or 'mersin' in text or 'onsekiz' in text or 'sakarya' in text or 'dokuz eylül' in text or 'karadeniz' in text or 'türk-alman' in text or 'necmettin' in text or 'atatürk' in text or 'koçman' in text or 'erciyes' in text or 'bursa teknik' in text or 'yalova' in text or 'konya teknik' in text:
        return 3
    elif 'yurtdışı' in text or 'kktc' in text:
        return 2
    else:
        return 1

df['Universite Adi'] = df['Universite Adi'].apply(etiketle)

In [20]:
# BURS ALIYOR MU?
df['Burs Aliyor mu?'] = df['Burs Aliyor mu?'].replace({
    'evet': 1, 
    'Evet': 1, 
    'EVET': 1,
    'Hayır': 0, 
    'hayır': 0
})

In [21]:
# BURS YUZDESİ
df['Burslu ise Burs Yuzdesi'].fillna(0, inplace=True)

In [22]:
# UNIVERSITE SINIF
df['Universite Kacinci Sinif'].fillna(0, inplace=True)
df['Universite Kacinci Sinif'] = df['Universite Kacinci Sinif'].replace({
    '1': 1, '2': 2, '3': 3,'4': 4, '5': 5,'6':6,'Yüksek Lisans':7,'tez':7,'Tez':7,'Mezun':0,"Hazırlık":0,"hazırlık":0,"0":0
})

In [23]:
# UNI NOT ORTALAMASI
df['Universite Not Ortalamasi'].fillna(0, inplace=True)
df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].replace({
    'ORTALAMA BULUNMUYOR': 0, 
    'Not ortalaması yok': 0, 
    'Ortalama bulunmuyor': 0,
    'Hazırlığım': 0,
    '0 - 1.79': 0,
    '1.00 - 2.50':1,
    '1.80 - 2.49':1,
    '2.50 ve altı':1,
    '2.00 - 2.50':1,
    '3.00-2.50':2,
    '2.50 - 3.00':2,
    '2.50 - 2.99':2,
    '2.50 -3.00':2,
    '3.00 - 3.50':3,
    '3.50-3':3,
    '3.00 - 3.49':3,
    '3.00 - 4.00':3,
    '3.50 - 4.00':4,
    '4-3.5':4,
    '4.0-3.5':4
})

In [24]:
# LISE TÜRÜ
df['Lise Turu'].fillna('Düz lise', inplace=True)
df['Lise Turu'] = df['Lise Turu'].replace({
    'İmam Hatip Lisesi': 0, 'Diğer': 0, 'Devlet': 0,'Düz lise': 0,
                   'Düz Lise': 0,'Meslek lisesi':1,'Meslek':1,'Meslek Lisesi':1,'Özel':1,"Özel Lisesi":1,
                   "Özel lisesi":1,"Özel Lise":1,'Anadolu Lisesi':2,'Anadolu lisesi':2,'Fen lisesi':3,'Fen Lisesi':3
})

In [25]:
# LISE NOTU
df['Lise Mezuniyet Notu'].fillna('3.00-2.50', inplace=True)

df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].replace({
    '75 - 100': 5,
    '100-75': 5,
    '84-70': 4,
    '100-85': 5,
    '4.00-3.50': 5,
    '50-75': 3,
    '50 - 75': 3,
    '3.00-4.00': 4,
    '3.00 - 4.00': 4, 
    '3.50-3.00': 4,
    '3.50-3': 4,
    '69-55': 3,
    '3.00-2.50': 3,
    '50-74': 3,
    '50 - 74': 3,
    '2.50 ve altı': 2,
    '54-45': 2,
    '25-50': 1,
    '25 - 50': 1,
    'Not ortalaması yok': 0,
    '44-0': 0,
    '0-25': 0,
    '0 - 25': 0,
    '25 - 49': 1,
    '0 - 24': 0
})
df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].astype(int)

In [26]:
# BASKA KURUMDAN BURS ALIYOR MU
df['Baska Bir Kurumdan Burs Aliyor mu?'].fillna('Hayır', inplace=True)

df['Baska Bir Kurumdan Burs Aliyor mu?'] = df['Baska Bir Kurumdan Burs Aliyor mu?'].replace({
    'Evet': 1, 
    'Hayır': 0
})

def burs_miktarini_doldur(row):
    if row['Baska Bir Kurumdan Burs Aliyor mu?'] == 0:
        return 0
    else:
        return row['Baska Kurumdan Aldigi Burs Miktari']

df['Baska Kurumdan Aldigi Burs Miktari'] = df.apply(burs_miktarini_doldur, axis=1)
df['Baska Kurumdan Aldigi Burs Miktari'].fillna(0, inplace=True)

In [27]:
# BURS MİKTARI
def temizle_ve_donustur(miktar):
    if pd.isna(miktar) or not isinstance(miktar, str):
        return 0

    temiz_miktar = re.sub(r'[^\d,]', '', miktar)
    temiz_miktar = temiz_miktar.replace(',', '.')

    try:
        miktar_numerik = float(temiz_miktar)
        if 100000 <= miktar_numerik < 1000000:
            miktar_numerik = int(str(int(miktar_numerik))[:3])
        return miktar_numerik
    except ValueError:
        return 0

df['Temizlenmiş Burs Miktarı'] = df['Baska Kurumdan Aldigi Burs Miktari'].apply(temizle_ve_donustur)

In [28]:
# ANNE ÇALIŞMA DURUMU
df['Anne Calisma Durumu'].fillna(2, inplace=True)
df['Anne Calisma Durumu'] = df['Anne Calisma Durumu'].replace({
    'Evet': 1, 
    'Hayır': 2,
    'Emekli':0
})

In [29]:
# ANNE EĞİTİM DURUMU
df['Anne Egitim Durumu'].fillna(0, inplace=True)
df['Anne Egitim Durumu'] = df['Anne Egitim Durumu'].replace({
    'İlkokul': 0,
    'İlkokul Mezunu': 0,
    'İLKOKUL MEZUNU': 0,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 1,
    'Ortaokul Mezunu': 1,
    'ORTAOKUL MEZUNU': 1,
    'Lise': 2,
    'LİSE': 2,
    'Lise Mezunu': 2,
    'Üniversite': 3,
    'ÜNİVERSİTE': 3,
    'Üniversite Mezunu': 3,
    'Yüksek Lisans': 4,
    'YÜKSEK LİSANS': 4,
    'Yüksek Lisans / Doktora': 4,
    'Yüksek Lisans / Doktara': 4,
    'Doktora': 5,
    'DOKTORA': 5
})

In [30]:
# ANNE SEKTÖR
df['Anne Sektor'].fillna(0, inplace=True)
df['Anne Sektor'] = df['Anne Sektor'].replace({
    '0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2
})

In [31]:
# BABA EĞİTİM DURUMU
df['Baba Egitim Durumu'].fillna(0, inplace=True)
df['Baba Egitim Durumu'] = df['Baba Egitim Durumu'].replace({
    'İlkokul': 0,
    'İlkokul Mezunu': 0,
    'İLKOKUL MEZUNU': 0,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 1,
    'Ortaokul Mezunu': 1,
    'ORTAOKUL MEZUNU': 1,
    'Lise': 2,
    'LİSE': 2,
    'Lise Mezunu': 2,
    'Üniversite': 3,
    'ÜNİVERSİTE': 3,
    'Üniversite Mezunu': 3,
    'Yüksek Lisans': 4,
    'YÜKSEK LİSANS': 4,
    'Yüksek Lisans / Doktora': 4,
    'Yüksek Lisans / Doktara': 4,
    'Doktora': 5,
    'DOKTORA': 5,
    '0':0
})

In [32]:
# BABA ÇALIŞMA DURUMU
df['Baba Calisma Durumu'].fillna(0, inplace=True)
df['Baba Calisma Durumu'] = df['Baba Calisma Durumu'].replace({
    'Evet': 0,
    'Hayır': 2,
    'Emekli': 1
})

In [33]:
# BABA SEKTÖR
df['Baba Sektor'].fillna(0, inplace=True)
df['Baba Sektor'] = df['Baba Sektor'].replace({
    '0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2
})

In [34]:
# KARDEŞ SAYISI
df['Kardes Sayisi'].fillna(0, inplace=True)
df['Kardes Sayisi'] = df['Kardes Sayisi'].replace({
    'Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat': 100,
    "0.0": 0,
    "1.0": 1,
    "2.0": 2,
    "3.0": 3,
    "4.0": 4,
    "5.0": 5,
    "6.0": 6,
    "7.0": 7,
    "8.0": 8,
    "9.0": 9,
    "10.0": 10,
    "11.0": 11,
    "12.0": 12,
    "13.0": 13,
    "14.0": 14,
    "18": 18,
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10
})
df['Kardes Sayisi'] = df['Kardes Sayisi'].astype(int)

In [35]:
# GIRISIMCILIK KULUBU UYELIGI
df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna(0, inplace=True)
df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [36]:
# SPOR DALI
df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna(0, inplace=True)
df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [37]:
# SPOR ROLU
df['Spor Dalindaki Rolunuz Nedir?'].fillna(0, inplace=True)
df['Spor Dalindaki Rolunuz Nedir?'] = df['Spor Dalindaki Rolunuz Nedir?'].replace({
    '0': 0, '-': 0,'Diğer':0,'DİĞER':0,'Bireysel Spor':1,'Bireysel':1,'Takım Oyuncusu':1,'Lider/Kaptan':2,'KAPTAN / LİDER':2,'Kaptan':2
})

In [38]:
# STK UYELİGİ
df['Aktif olarak bir STK üyesi misiniz?'].fillna(0, inplace=True)
df['Aktif olarak bir STK üyesi misiniz?'] = df['Aktif olarak bir STK üyesi misiniz?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [39]:
# STK PROJESİNE KATILDINIZ MI
df['Stk Projesine Katildiniz Mi?'].fillna(0, inplace=True)
df['Stk Projesine Katildiniz Mi?'] = df['Stk Projesine Katildiniz Mi?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [40]:
# GİRİŞİMCİLİK DENEYİMİ
df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna(0, inplace=True)
df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [41]:
# İNGİLİZCE
df['Ingilizce Biliyor musunuz?'].fillna(0, inplace=True)
df['Ingilizce Biliyor musunuz?'] = df['Ingilizce Biliyor musunuz?'].replace({
    'Hayır': 0, 'Evet': 1
})

In [42]:
# İNGİLİZCE
df['Ingilizce Seviyeniz?'].fillna(0, inplace=True)
df['Ingilizce Seviyeniz?'] = df['Ingilizce Seviyeniz?'].replace({
    '0': 0, 'Başlangıç': 1,'Orta':2,'İleri':3
})

In [43]:
def yas_grubu(yas):
    if yas < 18:
        return 4
    elif 18 <= yas <= 24:
        return 3
    elif 25 <= yas <= 34:
        return 2
    elif 35 <= yas <= 50:
        return 1
    else:
        return 0

df['Yas Grubu'] = df['yas'].apply(yas_grubu)

df['Tam Burslu'] = (df['Burslu ise Burs Yuzdesi'] == 100).astype(int)
df['Kismi Burslu'] = ((df['Burslu ise Burs Yuzdesi'] > 0) & (df['Burslu ise Burs Yuzdesi'] < 100)).astype(int)
df['Aile Gelir Skoru'] = df['Anne Calisma Durumu'] * 2 + df['Baba Calisma Durumu'] * 2 + df['Anne Egitim Durumu'] + df['Baba Egitim Durumu']
df['Universite Basari Skoru'] = df['Universite Not Ortalamasi'] + (df['Universite Kacinci Sinif'] * 0.5) + df['Universite Turu']
df['Aile Egitim Ortalamasi'] = (df['Anne Egitim Durumu'] + df['Baba Egitim Durumu']) / 2
df['Anne Calisma Egitim'] = df['Anne Calisma Durumu'] * df['Anne Egitim Durumu']
df['Baba Calisma Egitim'] = df['Baba Calisma Durumu'] * df['Baba Egitim Durumu']

In [44]:
df = df.drop(columns=['Dogum Yeri', 'Lise Adi Diger', 'Lise Bolum Diger','Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite',"Hangi STK'nin Uyesisiniz?"])

In [45]:
df = df.drop(columns=['Bölüm','Lise Bolumu','Daha Once Baska Bir Universiteden Mezun Olmus', 'Lise Adi', 'Lise Sehir','Burs Aldigi Baska Kurum','Baska Kurumdan Aldigi Burs Miktari','Uye Oldugunuz Kulubun Ismi' , 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'])

In [46]:
df.isnull().sum()

Basvuru Yili                                                0
Degerlendirme Puani                                     11050
Cinsiyet                                                    0
Ikametgah Sehri                                             0
Universite Adi                                              0
Universite Turu                                             0
Burslu ise Burs Yuzdesi                                     0
Burs Aliyor mu?                                             0
Universite Kacinci Sinif                                    0
Universite Not Ortalamasi                                   0
Lise Turu                                                   0
Lise Mezuniyet Notu                                         0
Baska Bir Kurumdan Burs Aliyor mu?                          0
Anne Egitim Durumu                                          0
Anne Calisma Durumu                                         0
Anne Sektor                                                 0
Baba Egi

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76174 entries, 0 to 76173
Data columns (total 39 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Basvuru Yili                                          76174 non-null  int32  
 1   Degerlendirme Puani                                   65124 non-null  float64
 2   Cinsiyet                                              76174 non-null  int64  
 3   Ikametgah Sehri                                       76174 non-null  int64  
 4   Universite Adi                                        76174 non-null  int64  
 5   Universite Turu                                       76174 non-null  int64  
 6   Burslu ise Burs Yuzdesi                               76174 non-null  float64
 7   Burs Aliyor mu?                                       76174 non-null  int64  
 8   Universite Kacinci Sinif                              76

In [48]:
train=df[:65125]
test=df[65125:]

In [49]:
train['Degerlendirme Puani'].fillna(0, inplace=True)

In [50]:
test.drop(columns=['Degerlendirme Puani'], inplace=True)

## MODELLİNG

In [52]:
x=train.drop(['Degerlendirme Puani'],axis=1) 
y=train[['Degerlendirme Puani']]

In [53]:
x=pd.get_dummies(data=x,drop_first=True)

In [54]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42) 

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree, ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

from sklearn.preprocessing import StandardScaler


def algo_test(x,y):
        #Bütün modelleri tanımlıyorum
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        sgd=SGDRegressor()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        rkn=RadiusNeighborsRegressor(radius=1.0)
        ada=AdaBoostRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
        svr=SVR()
        mlp_regressor = MLPRegressor()

       
        
        algos=[L,R,Lass,E,sgd,ETR,GBR,ada,kn,dt,xgb,svr,mlp_regressor]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','SGD','Extra Tree','Gradient Boosting',
                    'KNeighborsRegressor','AdaBoost','Decision Tree','XGBRegressor','SVR','mlp_regressor']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.20,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        #Hata ve doğruluk oranlarını bir tablo haline getirmek için bir dataframe oluşturuyorum
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
            

        #result adlı tabloya doğruluk ve hata oranlarımı yerleştiriyorum
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
       #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [56]:
#algo_test(x,y)

In [57]:
from xgboost import XGBRegressor

In [58]:
x=pd.get_dummies(data=x,drop_first=True)

In [59]:
model = XGBRegressor()

In [60]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [61]:
tahmin=model.predict(x_test)

In [62]:
r2_score(y_test,tahmin)

0.9103851184278279

In [63]:
mean_squared_error(y_test,tahmin)**.5

5.44119924005237

In [64]:
tahminler = model.predict(test)

In [65]:
predictions_df = pd.DataFrame({
    'id': test['id'], 
    'Degerlendirme Puani': tahminler
})

predictions_df.head()

,id,Degerlendirme Puani
65125,0,41.070244
65126,1,28.593136
65127,2,26.335018
65128,3,35.775871
65129,4,47.308025


In [66]:
predictions_df.to_csv('prediction.csv', index=False)